In [59]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import Any, Dict, Optional, Union
import os

class LanguageModelManager:
    """
    Hugging FaceのTransformersライブラリを使用して、自然言語生成（NLG）を簡単に行うためのラッパークラス。
    
    前提条件:
        - PyTorchがインストールされていること。
        - Hugging FaceのTransformersライブラリがインストールされていること。
    
    属性:
        tokenizer: Hugging FaceのAutoTokenizerインスタンス。
        model: Hugging FaceのAutoModelForCausalLMインスタンス。
    
    参考文献:
        - Tokenizerの詳細: https://huggingface.co/docs/transformers/main_classes/tokenizer
        - Modelの詳細: https://huggingface.co/docs/transformers/v4.33.0/en/main_classes/model
        
    
    使用例:
        >>> my_model = LanguageModelManager(model_name_or_path = "rinna/japanese-gpt-1b")
        >>> my_model.generate_text("こんにちは", only_answer=True)
    """
    
    def __init__(self, 
                 model_name_or_path: Union[str, os.PathLike], 
                 tokenizer_options: Optional[Dict[str, Any]] = None, 
                 model_options: Optional[Dict[str, Any]] = None) -> None:
        """
        コンストラクタで指定されたモデル名を使用して、tokenizerとmodelを初期化します。
        
        引数:
            model_name (str): 使用するモデルの名前。
            tokenizer_option (Optional[Dict[str, Any]]): Tokenizerの設定オプション。
            model_option (Optional[Dict[str, Any]]): モデルの設定オプション。
        
        戻り値:
            なし
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, **(tokenizer_options or {}))
        self.model = AutoModelForCausalLM.from_pretrained(model_name_or_path, **(model_options or {}))
        
        if torch.cuda.is_available():
            self.model = self.model.to("cuda")

    def generate_text(self, 
               text: str, 
               encode_options: Optional[Dict[str, Any]] = None, 
               generate_options: Optional[Dict[str, Any]] = None, 
               decode_options: Optional[Dict[str, Any]] = None,
               only_answer: bool = False) -> str:
        """
        テキストを入力として受け取り、モデルによって生成されたテキストを返します。
        
        処理のステップ:
            1. テキストをトークンにエンコード。
            2. トークンを使用してテキストを生成。
            3. 生成されたトークンをデコードしてテキストに変換。
        
        引数:
            text (str): 入力テキスト。
            encode_option (Optional[Dict[str, Any]]): エンコードオプション。
            generate_option (Optional[Dict[str, Any]]): テキスト生成オプション。
            decode_option (Optional[Dict[str, Any]]): デコードオプション。
            only_answer (bool): Trueの場合、生成されたテキストのうち、入力テキスト以降の部分のみを返します。
        
        戻り値:
            str: 生成されたテキスト。
        """
        
        encode_options = encode_options or {"return_tensors": "pt"}
        
        token_ids = self.tokenizer.encode(text, **encode_options)
        
        with torch.no_grad():
            output_ids = self.model.generate(
                input_ids=token_ids.to(self.model.device),
                pad_token_id=self.tokenizer.pad_token_id,
                bos_token_id=self.tokenizer.bos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                bad_words_ids=[[self.tokenizer.unk_token_id]],
                **(generate_option or {})
            )
        
        output = self.tokenizer.decode(output_ids.tolist()[0], **(decode_options or {}))
        output = output.replace("</s>", "")
        
        if only_answer:
            return output[len(text):]
        
        return output


In [51]:
m = LLM(model_name = "rinna/japanese-gpt-1b")
generate_option = {"min_length":50,"max_length":50, "do_sample": True, "top_k": 500,  "top_p":0.95}
m.output(text = '私の趣味は', generate_option= generate_option, only_answer=True )

'それについては、私が得意とする分野だからなのか、今もなお熱く語ることができるのですが、仕事となると話は別です。どんな人が相手でも、それは仕事です。もちろん、クライアントが要望をおっしゃってくださることは素晴らしいことです。'

In [54]:
m = LLM(model_name = "line-corporation/japanese-large-lm-1.7b")
m.output("趣味は？")

/Users/shibuya/python_env/venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


'趣味は?</s> ウェブ<0x0A>本文:  1000円札を1枚、'

In [60]:
m = LanguageModelManager(model_name_or_path = "rinna/japanese-gpt-1b")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'こんにちは このたび、東洋・東南アジア全域を対象とする統一的な診療指針の策定に向けた政府間協議に東洋医学界を代表して派遣されましたことを謹んでご報告いたします。各国保健省、衛生省との意見交換や関連資料の調査に協力しながら、'

In [61]:
m.generate_text(text="こんにちは", only_answer=True)

' このたび、平成25年1月3日より、表参道ヒルズ (東京都港区南青山)にて、 「COCHA」の新プロモーションイベントを開催いたします。 今回、プロジェクトテーマを「#Bararaness... 続きを読む →...'